In [1]:
import pandas as pd
import numpy as np
import pydicom as dicom
import os, cv2, sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
output_dir = 'patches'

class_names = {'non-tumor': 0,
               'adenocarcinoma': 1,
               'hematopoietic ': 2,
               'squamous cell carcinoma ': 3,
               'melanoma': 4,
               'neuroendocrine carcinoma ': 5,
               'sarcoma ': 6,
               'urothelial carcinoma ': 7,}

excel_path = 'Mets_morphology_dataset_fin.xlsx'
excel = pd.read_excel(excel_path)
excel['number'] = excel['number'].astype('str')

# class_names = {j:i for i,j in enumerate(excel['Histomorphology'].unique().tolist())}

n_classes = len(class_names)
class_labels = {excel['number'][i]:class_names[excel['Histomorphology'][i]]  for i in range(len(excel))}

In [3]:
image_dir = 'Channel Images'

patch_size = 224
step_size = 200

if not(os.path.exists(output_dir)):
    os.mkdir(output_dir)
folders = os.listdir(image_dir)

crop = np.zeros((patch_size,patch_size,3))

for i, patient in enumerate(folders):
    image_subdir = os.path.join(image_dir,patient)
    files = os.listdir(image_subdir)

    temp_df = pd.DataFrame({'col1': files})
    temp_df['number'] = temp_df['col1'].apply(lambda x: int(x.split('_')[1]))
    temp_df['name'] = temp_df['col1'].apply(lambda x: x.split('_')[0])

    for sub_df in temp_df.groupby('name'):
        sorted_files = sub_df[1].sort_values(by=['number']).reset_index(drop=True)['col1'].tolist()
        for j in range(0,len(sorted_files),2):
            # read dcm file
            splited_name = sorted_files[j].split('_')
            label = class_labels[patient]
            if label not in [6,7]:
                dcm_sample1 = (dicom.dcmread(os.path.join(image_subdir, sorted_files[j])).pixel_array/65536)*255
                dcm_sample2 = (dicom.dcmread(os.path.join(image_subdir, sorted_files[j+1])).pixel_array/65536)*255
                file_shape1 = dcm_sample1.shape[0]
                file_shape2 = dcm_sample1.shape[1]
                counter = 0
                for m in range(0, file_shape1-(patch_size-step_size)+1, step_size):
                    for n in range(0, file_shape2-(patch_size-step_size)+1, step_size):
                        if m+patch_size>file_shape1:
                            o= file_shape1-patch_size
                        else:
                            o=m
                        if n+patch_size>file_shape2:
                            p= file_shape2-patch_size
                        else:
                            p=n
                        if np.max(dcm_sample1[o:o+patch_size, p:p+patch_size])>5 and np.max(dcm_sample2[o:o+patch_size, p:p+patch_size])>5:

                            crop[:,:,0] = dcm_sample2[o:o+patch_size, p:p+patch_size] - dcm_sample1[o:o+patch_size, p:p+patch_size]
                            crop[:,:,1] = dcm_sample1[o:o+patch_size, p:p+patch_size]
                            crop[:,:,2] = dcm_sample2[o:o+patch_size, p:p+patch_size]
                            
                            counter+= 1
                            if counter<10:
                                # cv2.imwrite('{}/{}_{}_{}_label_{}_patch_0{}_.png'.format(output_dir, patient, splited_name[0], splited_name[1], label, counter), cv2.resize(crop, (224,224)))
                                cv2.imwrite('{}/{}_{}_{}_label_{}_patch_0{}_.png'.format(output_dir, patient, splited_name[0], splited_name[1], label, counter), crop)
                            else:
                                cv2.imwrite('{}/{}_{}_{}_label_{}_patch_{}_.png'.format(output_dir, patient, splited_name[0], splited_name[1], label, counter), crop)
    sys.stdout.write("\rfolder {}/{}".format(i+1,len(folders)))
    sys.stdout.flush()

folder 111/111

In [6]:
crop.shape

(224, 224, 3)